
# hate speech detection


### Install and import necessary packages

In [1]:

import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


# Load data

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!ls '/content/drive/MyDrive/latsis'
df_train = pd.read_csv('/content/drive/MyDrive/latsis/train.csv')
df_dev = pd.read_csv('/content/drive/MyDrive/latsis/dev.csv')
extra_data = pd.read_parquet('/content/drive/MyDrive/latsis/german_datasets.parquet')
#df_test = pd.read_csv('/content/drive/MyDrive/latsis/test.csv')

Mounted at /content/drive
dev.csv  german_datasets.parquet  train.csv


### split train/val set

In [3]:
#convert to string

extra_data["text"] = extra_data["text"].astype(str)
df_train["text"] = df_train["text"].astype(str)
df_dev["text"] = df_dev["text"].astype(str)

#merge train and dev
full = pd.concat([df_train, df_dev])

#split train and test
df_train, df_dev = train_test_split(full, test_size=0.2, random_state=42)

train, dev = train_test_split(full, test_size=0.2, random_state=42)
train = shuffle(train, random_state = 42)
dev = shuffle(dev, random_state = 42)
# X_train, y_train = shuffle(, random_state=42) # shuffle the training data
# X_val, y_val = shuffle(X_val, y_val, random_state=42)

## ML approaches
### tf-idf

In [4]:
from zmq.constants import XPUB_NODROP
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
#tf-idf
vectorizer = TfidfVectorizer()
scaler =  StandardScaler(with_mean=False)

# Fit the tfidf model
vectorizer = vectorizer.fit(train['text'])

x_train = vectorizer.transform(train['text'])
x_dev = vectorizer.transform(dev['text'])
y_train = train['label']
y_dev = dev['label']

# Fit the scaler
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_dev = scaler.transform(x_dev)

print("X_train shape is {}".format(x_train.shape))
print("X_dev shape is {}".format(x_dev.shape))

X_train shape is (7736, 23641)
X_dev shape is (1934, 23641)


### models fit

In [6]:
!pip3 install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.2 MB/s eta 0:00:00


In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture
from sklearn.kernel_approximation import Nystroem
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process.kernels import *

from tqdm import tqdm
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRFClassifier, XGBRegressor, XGBRFRegressor


In [10]:
# SVM model

from sklearn.svm import LinearSVC
from joblib import dump, load
import matplotlib.pyplot as plt

# define model
model = LinearSVC(class_weight='balanced', random_state=42)

model.fit(x_train, y_train)
from sklearn.metrics import f1_score, accuracy_score

# Plotting scores
y_pred = model.predict(x_dev)

f1 = f1_score(y_dev, y_pred, average="macro")
print("Test f1 score : %s "% f1)
acc = accuracy_score(y_dev, y_pred)
print("Test accuracy score : %s "% acc)


# from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# # Plot confusion matrix
# cm = confusion_matrix(y_dev, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
# disp.plot(cmap=plt.cm.Blues)
# plt.show()

Test f1 score : 0.5997503713947436 
Test accuracy score : 0.5997931747673216 


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [11]:
models = [XGBClassifier(),
          LGBMClassifier(),
          CatBoostClassifier(logging_level='Silent'),
          ExtraTreesClassifier(),
          RandomForestClassifier(),
          AdaBoostClassifier()]
          # GaussianProcessClassifier(kernel=RationalQuadratic())

for model in models:
           model.fit(x_train, y_train)
           y_pred = model.predict(x_dev)
           f1 = f1_score(y_dev, y_pred, average="macro")
           print("Test f1 score : %s "% f1)
           acc = accuracy_score(y_dev, y_pred)
           print("Test accuracy score : %s "% acc)

Test f1 score : 0.6416675244467318 
Test accuracy score : 0.6416752843846949 
[LightGBM] [Info] Number of positive: 3863, number of negative: 3873
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035746 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30270
[LightGBM] [Info] Number of data points in the train set: 7736, number of used features: 818
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499354 -> initscore=-0.002585
[LightGBM] [Info] Start training from score -0.002585
Test f1 score : 0.6401225557034922 
Test accuracy score : 0.640124095139607 
Test f1 score : 0.6592152495444414 
Test accuracy score : 0.6592554291623578 
Test f1 score : 0.6745154789469171 
Test accuracy score : 0.6752843846949328 
Test f1 score : 0.637793994732522 
Test accuracy score : 0.6380558428128231 
Test f1 score : 0.6112844095703627 
Test accuracy sc